In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pymongo

In [2]:
# Database setup
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['food_analysis']
recipes = db['recipes']

In [3]:
choco_chip = pd.DataFrame(list(recipes.find({"title":{"$regex":"chocolate chip", "$options" :'i'}})))

In [4]:
choco_chip.to_csv('choco_chip.csv')

In [5]:
choco_chip

,_id,title,total_time,yields,ingredients,instructions,image,rating,author,reviews,...,prepTime,cookTime,totalTime,datePublished,recipeYield,recipeCategory,cookingMethod,recipeCuisine,review_count,keywords
0,5f20f9eb3aa0b95774cf6208,My Kids Favourite Chocolate Chip Cookies,18,,"[2⁄3 cup butter, melted, 2 cups lightly packed...","Preheat oven to 375°F.\nBeat the butter, brown...","https://img.sndimg.com/food/image/upload/q_92,...",4.5,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5f217eba61c02dddcf3f5942,Secret Ingredient Chocolate Chip Cookies Jorge...,0,30 item(s),"[1 1/2 cups all-purpose flour, 1 teaspoon baki...",Preheat oven to 350° F.\nInto a bowl sift toge...,None,3,None,[{'review_text': 'I did not personally sample ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5f217f5b61c02dddcf3f5992,Large Batch Chocolate Chunk (Chocolate Chip) C...,36,72 serving(s),"[1 1⁄2 cups brown sugar, 1 1⁄2 cups white suga...",Preheat Oven to 325°F.\nIn a VERY large mixing...,"https://img.sndimg.com/food/image/upload/q_92,...",5,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5f217f6061c02dddcf3f5995,"Chocolate Chip, Pecan & Sea Salt Cookies",30,,[1⁄2 cup room temperature butter (1 stick I us...,Grease or spray your cookie sheet or tray. Spo...,"https://img.sndimg.com/food/image/upload/q_92,...",4,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5f217fa761c02dddcf3f59bd,Chocolate Chip Cookies - Betty Crocker's 1969 ...,18,,"[2⁄3 cup shortening, 2⁄3 cup butter or 2/3 cup...",Heat oven to 375 degrees.\r\nMix thorougly sho...,"https://img.sndimg.com/food/image/upload/q_92,...",5,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5f217ffc61c02dddcf3f59eb,Different Chocolate Chip Cookies,27,48 serving(s),"[3⁄4 cup brown sugar, 3⁄4 cup white sugar, 1 c...","Mix sugars and oil.\nAdd eggs, one at a time, ...","https://img.sndimg.com/food/image/upload/q_92,...",5,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5f21805361c02dddcf3f5a1b,Scrumptious Giant White Chocolate Chip Macadam...,40,16 serving(s),"[1 cup unsalted macadamia nuts, roughly choppe...",Heat oven to 350 degrees. Spread macadamia nut...,"https://img.sndimg.com/food/image/upload/q_92,...",-1,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5f2180c361c02dddcf3f5a52,Claudia's Chocolate Chip Cookies,17,,"[1 cup butter, 3⁄4 cup sugar, 3⁄4 cup brown su...",Preheat the oven to 325°F.\r\nCream the butter...,"https://img.sndimg.com/food/image/upload/q_92,...",3,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5f21820461c02dddcf3f5aef,Chocolate Chip Cookies (With Special Ingredient),33,,"[nonstick cooking spray, 3⁄4 cup of firmly pac...",Preheat oven to 350 degrees.\nCoat a baking sh...,"https://img.sndimg.com/food/image/upload/q_92,...",-1,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5f219f0461c02dddcf3f5dc9,Ultimate Chocolate Chip Cookies,45,,[3⁄4 cup Butter Flavor Crisco or 3/4 cup butte...,b'Preheat oven to 350\xc2\xb0.\nCombine Butter...,"https://img.sndimg.com/food/image/upload/q_92,...",3.5,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Ingredient Reference
ir = {"flour":{
    "types":['all-purpose', "white", "brown", "spelt", "oat", "millet"
             "whole-wheat", "almond", "tapioca", "sorghum", "rice", "coconut", "garfava"
            "barley", "cake", "bread", "self-rising", "soy", "pastry"]
    "common_units":["cup", "c.", "oz", "gram", "g"]
}}

# Quantities
units = ["cup", "g", 'teaspoons', "tsp", "tbsp", "g", "oz", 'tablespoons','cups','containers','packets','bags','quarts','pounds','cans','bottles',
        'pints','packages','ounces','jars','heads','gallons','drops','envelopes','bars','boxes','pinches',
        'dashes','bunches','recipes','layers','slices','links','bulbs','stalks','squares','sprigs',
        'fillets','pieces','legs','thighs','cubes','granules','strips','trays','leaves','loaves','halves']

fractions = ["↉", "⅒", "⅑", "⅛", "⅐", "⅙", "⅕", "¼", "⅓", "½", "⅖", "⅔", "⅜", "⅗", "¾", "⅘", "⅝", "⅚", "⅞"]

In [ ]:
# The staples
choco_chip['flour'] = 

In [13]:
for recipe in choco_chip.ingredients:
    for ing_ in recipe:
        ing = ing_.lower()
        split=[]
        if "flour" in ing:
            for unit in ["cup", "c.", "oz", "gram", "g"]:
                if unit+"s" in ing:
                    split = ing.split(" "+"cup"+"s ")
                else:
                    split = ing.split(" "+"cup"+" ")
                
        if len(split) > 1:
            quant = split[0]
            desc = split[1]
            
            
        

SyntaxError: can't assign to function call (<ipython-input-13-9e2c56ef5d13>, line 2)